In [10]:
from pymc import Model, Binomial, Normal, MvNormal, math, Exponential, Beta, sample
import numpy as np
import pandas as pd
import pytensor.tensor as pt
from pytensor.tensor.linalg import cholesky
from utility import data_loader, data_cleaner, sample_dist
import arviz as az

In [17]:
berlin_data, vanautu_data, adult_data, child_data = data_loader()

In [18]:
d_berlin, d_vanautu, MA, pop_berlin, pop_berlin_raw, pop_vanautu_raw, pop_vanautu = data_cleaner(berlin_data, vanautu_data, adult_data, child_data)

/home/harrison/Desktop/gitHubRepos/CausalAnalysis/generalisability/utility.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adult_data["choice"] = adult_data["T1_ad_choice_1yes"]
/home/harrison/Desktop/gitHubRepos/CausalAnalysis/generalisability/utility.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child_data["choice"] = child_data["T1_choice_1yes"]
/home/harrison/Desktop/gitHubRepos/CausalAnalysis/generalisability/utility.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [4]:
# Create matrices with zeros
sample_berlin, sample_vanuatu = sample_dist(d_berlin, d_vanautu)

In [5]:
#Covaraince Kenrel defined for the paper and implemented in Stan.
MA = 20
age = d_berlin["Age_binned"].values - 1
outcome = d_berlin["choice"]
gender = d_berlin["gender"].values - 1
d_berlinf = d_berlin[d_berlin["gender"] == 1]
d_berlinm =d_berlin[d_berlin["gender"] == 2]
d_berlinf['Age_binned'] = d_berlinf["Age_binned"].astype(int)
d_berlinm['Age_binned'] = d_berlinm["Age_binned"].astype(int)

/tmp/ipykernel_204645/3536775231.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_berlinf['Age_binned'] = d_berlinf["Age_binned"].astype(int)
/tmp/ipykernel_204645/3536775231.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_berlinm['Age_binned'] = d_berlinm["Age_binned"].astype(int)


In [6]:
P_pop = pop_berlin_raw
P_other = pop_vanautu_raw


def GPL(K, C, D, S, epsilon=1e-6):
    KR = np.arange(1, K + 1)
    Rho = C * pt.exp(-D * ((KR[:, None] - KR[None, :]) ** 2) / K ** 2)
    pt.fill_diagonal(Rho, 1)
    
    # Add a small diagonal perturbation
    Rho += epsilon * pt.eye(K)
    
    return S * cholesky(Rho)
    
with Model() as gp:
    # Priors for parameters
    alpha = Normal('alpha', mu=0, sigma=3, shape=2)
    eta = Exponential('eta', lam=2, shape=2)
    sigma = Exponential('sigma', lam=1, shape=2)
    rho = Beta('rho', alpha=10, beta=1, shape=2)

    # Gaussian process age effects
    age_effectf = MvNormal("age_effectf",
        mu=np.zeros(MA),
        chol=GPL(MA, rho[0], eta[0], sigma[0])#, shape = MA
    ) 

    age_effectm = MvNormal("age_effectm",
        mu=np.zeros(MA),
        chol=GPL(MA, rho[1], eta[1], sigma[1])#, shape = MA
    ) 
      
    # Linear model
    pf = math.invlogit(alpha[0] + age_effectf[ np.array(d_berlinf['Age_binned']) ] ) 
    pm = math.invlogit(alpha[1] + age_effectm[ np.array(d_berlinm['Age_binned']) ] ) 
    
    # Likelihood function
    outcome_obsf = Binomial('outcome_obsf', n=1, p = pf, observed = d_berlinf["choice"].values)
    outcome_obsm = Binomial('outcome_obsm', n=1, p = pm, observed = d_berlinm["choice"].values)

In [21]:
with gp:
     trace = sample(2000, tune=2000, chains=4, cores=8, init='adapt_diag', target_accept=0.99, nuts_sampler = "numpyro")

Compiling...
Compilation time = 0:00:03.851074
Sampling...
Compiling.. :   0%|                                    | 0/4000 [00:00<?, ?it/s]

  0%|                                                  | 0/4000 [00:00<?, ?it/s]

Compiling.. :   0%|                                    | 0/4000 [00:00<?, ?it/s]


  0%|                                                  | 0/4000 [00:00<?, ?it/s]


Compiling.. :   0%|                                    | 0/4000 [00:00<?, ?it/s]

Running chain 2:   0%|                                 | 0/4000 [00:04<?, ?it/s]


Running chain 0:   0%|                                 | 0/4000 [00:04<?, ?it/s]


Running chain 0:   5%|█▏                     | 200/4000 [00:15<03:29, 18.14it/s]

Running chain 2:   5%|█▏                     | 200/4000 [00:16<03:46, 16.79it/s]


Running chain 0:  10%|██▎                    | 400/4000 [00:31<04:11, 14.31it/s]

Running chain 2:  10%|██▎                    | 400/4000 [00:33<04:21, 13.78it/s]


Running chain 3:  10%|██▎        

In [22]:
az.summary(trace)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],-0.374,0.379,-1.014,0.331,0.137,0.100,8.0,29.0,1.48
alpha[1],1.142,1.193,-0.622,2.998,0.574,0.437,5.0,27.0,2.15
age_effectf[0],-0.065,0.257,-0.705,0.313,0.093,0.068,7.0,16.0,1.50
age_effectf[1],-0.047,0.236,-0.666,0.270,0.085,0.063,8.0,16.0,1.49
age_effectf[2],-0.028,0.218,-0.602,0.268,0.079,0.058,8.0,17.0,1.49
age_effectf[3],-0.009,0.203,-0.527,0.302,0.074,0.054,8.0,16.0,1.50
age_effectf[4],0.011,0.193,-0.495,0.321,0.070,0.052,7.0,15.0,1.53
age_effectf[5],0.032,0.188,-0.472,0.343,0.070,0.051,7.0,15.0,1.56
age_effectf[6],0.053,0.187,-0.438,0.401,0.071,0.053,7.0,15.0,1.63
age_effectf[7],0.074,0.192,-0.260,0.573,0.075,0.056,6.0,13.0,1.75


TypeError: 'RangeIndex' object is not callable